# Introduction and setup

In this notebook, I have code to load in the stellar profiles in 3 dimensions which 
were previously calculated on the command line so that we can plot quantities such as 
the 1D velocity dispersion and evolution with time of the mass.

In [ ]:
using Revise
using LilGuys
using CairoMakie, Arya

In [ ]:
modelsdir = ENV["DWARFS_ROOT"] * "/analysis"

In [ ]:
import TOML
obs_props = TOML.parsefile(joinpath(ENV["DWARFS_ROOT"], "observations/sculptor/observed_properties.toml"))

In [ ]:
function load_profile(modelname, starsname)
    orbit = load_profiles(modelname * "/stars/" * starsname)

    return orbit
end

In [ ]:
function load_isolation(modelname, starsname)
    iso = load_profiles(modelname * "/../stars/" * starsname)
    return iso
end

In [ ]:
function load_profiles(filename; extension="stellar_profiles_3d.hdf5")
    path = joinpath(modelsdir, filename, extension)
    
    profs = LilGuys.read_structs_from_hdf5(path, LilGuys.StellarProfile3D)
    idxs = parse.(Int, first.(profs))
    profs = last.(profs)

    s = sortperm(idxs)
    return [idxs[i] => profs[i] for i in s ]
end

In [ ]:
function load_prof_expected(filename, parampath="profile.toml")
    path = joinpath(modelsdir, filename, parampath)
        
    if isfile(parampath)
        expected = LilGuys.load_profile(parampath)
    else
        error("file not found $path")
    end

    return expected
end

In [ ]:
function compare_profiles(profiles, expected=nothing; 
        sequential=false, legend=true, limits=(-1.5, 0.8, -15, 3), 
        kwargs...
    )
    
    fig = Figure()

	ax = Axis(fig[1,1], xlabel=L"\log\, r / \textrm{kpc}", ylabel =  L"\log\, \rho_\star\; [10^{10} M_\odot / \textrm{kpc}^3]", 
		limits=limits; kwargs...
		)

    plot_kwargs = Dict{Symbol, Any}()

    if sequential
        plot_kwargs[:colorrange] = (1, length(profiles))
    end
    
    for i in eachindex(profiles)
        label, profs = profiles[i]
        prof = profs[end].second
        x = prof.log_r
        y = log10.(prof.rho)
        if sequential
            plot_kwargs[:color] = i
        end

    	lines!(x, y, label=string(label); plot_kwargs...)
        arrows!([log10.(prof.r_break)],[-10], [0], [3], color=COLORS[i])
    end

    if expected !== nothing
    	log_r_pred = LinRange(-2, 2, 1000)
    	ρ_s_pred = calc_ρ.(expected, 10 .^ log_r_pred)

    	lines!(log_r_pred, log10.(ρ_s_pred), label="expected", color="black", linestyle=:dot)
    end

    if legend
    	axislegend(ax, position=:lb)
    end 
    
	fig
end

In [ ]:
function plot_v_t!(profiles; kwargs...)
	sigmas = [p.second.sigma_vx for p in profiles]

    t = [p.second.time for p in profiles] 
	scatterlines!(t * T2GYR, sigmas * V2KMS; kwargs...)
end

In [ ]:
function plot_v_t(profiles; kwargs...)
	fig, ax = FigAxis(
		xlabel = "time / Gyr",
		ylabel = L"\sigma_v / \textrm{km s^{-1}}";
        kwargs...
	)

    plot_v_t!(profiles)

    fig
end
    

In [ ]:
function compare_v_t(profilesi, profs_iso=nothing)
	fig, ax = FigAxis(
		xlabel = "time / Gyr",
		ylabel = L"\sigma_v / \textrm{km s^{-1}}",
	)


    for i in eachindex(profilesi)
        label, profiles = profilesi[i]

        kwargs = Dict{Symbol, Any}()
        kwargs[:label] = label
        kwargs[:color] = COLORS[i]
  
        plot_v_t!(profiles; kwargs...)
    end

    if profs_iso !== nothing
        for i in eachindex(profs_iso)
            label, profiles = profs_iso[i]
    
      
            sigmas = [p.second.sigma_vx for p in profiles]
    
            t = [p.second.time for p in profiles] 
            t = t .- t[end]
            scatterlines!(t * T2GYR, sigmas * V2KMS, color=COLORS[i])
            
        end
    end
    
    hlines!(obs_props["sigma_v"], color=:grey)
    hspan!(obs_props["sigma_v"] - obs_props["sigma_v_err"], 
        obs_props["sigma_v"] + obs_props["sigma_v_err"],
        color = (:grey, 0.1), label="observed")

    LilGuys.Plots.hide_grid!(ax)
    axislegend()
	fig
end

## 1e7 mean orbit

In [ ]:
profs = [
    "exp2d" => load_profile("sculptor/1e7_V31_r3.2/orbit_mean", "exp2d_rs0.13"),
   # "plummer" => load_profile("sculptor/1e7_V31_r3.2/orbit_mean", "plummer_rs0.18"),

];

In [ ]:
profs_iso = [
    "exp2d" => load_isolation("sculptor/1e7_V31_r3.2/orbit_mean", "exp2d_rs0.13"),
    #"plummer" => load_isolation("sculptor/1e7_V31_r3.2/orbit_mean", "plummer_rs0.18"),
];

In [ ]:
compare_v_t(profs, profs_iso)

In [ ]:
compare_profiles(profs)

In [ ]:
compare_profiles(profs_iso)

### Comparison between orbits

In [ ]:
profs = [
    "mean" => load_profile("sculptor/1e6_V31_r3.2/orbit_mean", "plummer_rs0.20"),
    "smallperi" => load_profile("sculptor/1e6_V31_r3.2/orbit_smallperi", "plummer_rs0.20"),
    "1e7" => load_profile("sculptor/1e7_V31_r3.2/orbit_mean", "plummer_rs0.20"),

];

In [ ]:
profs_iso = [
    "mean" => load_isolation("sculptor/1e6_V31_r3.2/orbit_mean", "plummer_rs0.20"),
    "exp2d" => load_isolation("sculptor/1e6_V31_r3.2/orbit_smallperi", "plummer_rs0.20"),
    "mean (1e7)" => load_isolation("sculptor/1e7_V31_r3.2/orbit_mean", "plummer_rs0.20"),
];

In [ ]:
compare_v_t(profs, profs_iso)

## Scratch (check that iso & initial are the same)

In [ ]:
weights = LilGuys.read_hdf5_table("/astro/dboyea/dwarfs/analysis/sculptor/1e6_V31_r3.2/stars/plummer_rs0.20/probabilities_stars.hdf5")

In [ ]:
snap_iso = Output("/astro/dboyea/dwarfs/analysis/isolation/1e6/fiducial/", weights=weights.probability)[end-1]

In [ ]:
snap_ini = Output("/astro/dboyea/dwarfs/analysis/sculptor/1e6_V31_r3.2/orbit_smallperi/", weights=weights.probability)[1]

In [ ]:
snap_ini0 = Snapshot("/astro/dboyea/dwarfs/simulations/sculptor/1e6_V31_r3.2/orbit_smallperi/initial.hdf5")
snap_ini0.weights = weights.probability[snap_ini0.index];

In [ ]:
m_scale = 0.5267
r_scale = 0.53613
v_scale = sqrt(m_scale / r_scale )

In [ ]:
snap_iso_scaled = LilGuys.rescale(snap_iso, m_scale, r_scale)

In [ ]:
prof_ini = LilGuys.MassProfile3D(snap_ini);
prof_iso2 = LilGuys.MassProfile3D(snap_iso)

In [ ]:
prof_iso = LilGuys.MassProfile3D(snap_iso_scaled)


In [ ]:
fig, ax = FigAxis()
scatter!(log10.(prof_ini.r_circ), log10.(prof_ini.v_circ))
scatter!(log10.(prof_iso.r_circ), log10.(prof_iso.v_circ))
scatter!(log10.(prof_iso2.r_circ * r_scale), log10.(prof_iso2.v_circ * v_scale))

fig

In [ ]:
cen = LilGuys.Centres.SS_State(snap_ini)
LilGuys.Centres.calc_centre!(cen, snap_ini)
cen.centre

In [ ]:
snap_ini.v_cen = cen.centre.velocity
snap_ini.x_cen = cen.centre.position

In [ ]:
snap_iso._radii = nothing

In [ ]:
LilGuys.calc_σv_1d(snap_iso, r_max=1/r_scale) * v_scale

In [ ]:
snap_iso_scaled._radii = nothing
LilGuys.calc_σv_1d(snap_iso_scaled)

In [ ]:
snap_iso_scaled.positions .- snap_ini0.positions

In [ ]:
LilGuys.calc_σv_1d(snap_ini)

In [ ]:
ss = LilGuys.Centres.SS_State(snap_ini0)
LilGuys.Centres.calc_centre!(ss, snap_ini0)
snap_ini0.x_cen = ss.centre.position
snap_ini0.v_cen = ss.centre.velocity

In [ ]:
snap_ini0.velocities

In [ ]:
snap_iso_scaled.positions .- snap_ini0.positions

In [ ]:
snap_ini0._radii = nothing

In [ ]:
LilGuys.calc_σv_1d(snap_ini)

In [ ]:
LilGuys.StellarProfile3D(snap_iso_scaled)

In [ ]:
profs_iso[1].second[end].second.sigma_vx

In [ ]:
profs[1].second[1].second.sigma_vx

In [ ]:
idx0 = sortperm(snap_ini0.index)
idx = sortperm(snap_ini.index)
snap_ini.positions[:, idx] .- snap_ini0.positions[:, idx0]